In [1]:
from library import *
from dataset_creation import *
from k_fold import *

In [2]:
def plot_color(values, size=(30,5), coef=1):
    
    my_cmap = plt.get_cmap('copper')
    my_norm = plt.Normalize(vmin=0, vmax=max(values)*coef)
    colormap=my_cmap(my_norm(values))
    
    return colormap

# Logistic Regression

In [3]:
clf = LogisticRegression(solver="liblinear")

In [4]:
clf.fit(X_train, y_train)
y_LR = clf.predict(X_test)

NameError: name 'X_train' is not defined

In [ ]:
mat = confusion_matrix(y_test, y_LR)

sns.set(font_scale=1.5) 
sns.heatmap(mat, square = True, annot=True, cbar=False, fmt="d")
plt.xlabel('predicted value')
plt.ylabel('true value');

In [ ]:
accuracy_score(y_test, y_LR)

In [ ]:
y_LR

# NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

clf = MultinomialNB()
y_pred = clf.fit(X_train, y_train)
y_MNB = clf.predict(X_test)

mat = confusion_matrix(y_test, y_MNB)

sns.set(font_scale=1.5) 
sns.heatmap(mat, square = True, annot=True, cbar=False, fmt="d")
plt.xlabel('predicted value')
plt.ylabel('true value');

In [ ]:
accuracy_score(y_test, y_MNB)

# GNB

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train)
y_GNB = gnb.predict(X_test)

mat = confusion_matrix(y_test, y_GNB)

sns.set(font_scale=1.5) 
sns.heatmap(mat, square = True, annot=True, cbar=False, fmt="d")
plt.xlabel('predicted value')
plt.ylabel('true value');

In [ ]:
accuracy_score(y_test, y_GNB)

# K-fold

In [ ]:
def apply_Logistic_Regression(X_tr, y_tr, X_te):
    clf = LogisticRegression(random_state=0)
    clf.fit(X_tr, y_tr)
    y_LR = clf.predict(X_te)
    return y_LR

In [ ]:
accuracies = []

for k in tqdm(range(2,10)):
    
    for f in tqdm(range(4,17)):

        kfold = KFold(n_splits=k, random_state=123, shuffle=True)

        model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)

        sfs_kn = sfs(estimator = model, n_features_to_select=f, cv=kfold, n_jobs=-1)



        # K-FOLD CROSS VALIDATION
        # using X_train and y_train as new dataset to split in X_train_new + X_validation
        sfs_kn.fit(X_res_train, y_res_train)

        temp_X_train = sfs_kn.transform(X_res_train)

        for train_index, test_index in kfold.split(temp_X_train, y_res_train):

            X_train, X_test = temp_X_train[train_index], temp_X_train[test_index]
            y_train, y_test = y_res_train[train_index], y_res_train[test_index]

            model = KNeighborsClassifier(n_neighbors=3, n_jobs=-1).fit(X_train, y_train)

            out = model.predict(X_test)

            accuracies.append((accuracy_score(y_test, out), k, f))




In [ ]:
sfs_kn.get_support()

In [ ]:
y_pred.shape

In [ ]:
X_res[:,sfs_kn.get_support()].shape

In [16]:
accuracies

[0.9127764127764127,
 0.9072481572481572,
 0.9060196560196561,
 0.8955773955773956]

In [18]:
accuracies

[0.8955773955773956,
 0.9078624078624079,
 0.9127764127764127,
 0.9103194103194103]